In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp

import warnings
warnings.filterwarnings('ignore')

from geopy.distance import geodesic

df = pd.read_csv("data_hackathon_v04.csv", sep="|", parse_dates=["created_at"], index_col="ride_id")

In [2]:
from h3 import h3

df["cell_id"] = df[["pickup_lat","pickup_lng"]].apply(lambda x: h3.geo_to_h3(*x, 7), axis=1)

In [3]:
def make_df_time(df):
    df_time = df[["created_at"]]

    df_time["day"] = df_time.created_at.dt.day
    df_time["month"] = df_time.created_at.dt.month
    df_time["year"] = df_time.created_at.dt.year
    df_time["is_year_start"] = np.int8(df_time.created_at.dt.is_year_start)
    df_time["is_year_end"] = np.int8(df_time.created_at.dt.is_year_end)
    df_time["dayofyear"] = df_time.created_at.dt.dayofyear
    df_time["dayofweek"] = df_time.created_at.dt.dayofweek

    df_time["week"] = df_time.created_at.dt.week

    df_time["hour"] = df_time.created_at.dt.hour
    df_time["minute"] = df_time.created_at.dt.minute
    df_time["final_time"] = (df_time["hour"] + df_time["minute"] // 30) % 24
    df_time["dayofweek"] = df_time.created_at.dt.dayofweek
    df_time["is_weekend"] = df_time["dayofweek"] // 5

    df_time.drop(["created_at"], inplace=True, axis=1)
    return df_time

def in_square(x, y):
    square = (49.895468, 23.907734, 49.769849, 24.121698)
    return (square[2] <= x <= square[0]) and (square[1] <= y <= square[3])

def clear_df(df):
    df["in_Lviv"] = (df[["pickup_lat", "pickup_lng"]].apply(lambda x: in_square(*x), axis=1) & df[["dropoff_lat", "dropoff_lng"]].apply(lambda x: in_square(*x), axis=1))
    df = df[df["in_Lviv"]]
    df["simple_distance"] = np.array(list(map(lambda x: geodesic(x[:2], x[2:]).km, 
                                          df[["pickup_lat", "pickup_lng", "dropoff_lat", "dropoff_lng"]].values)))
    df = df[abs(df["simple_distance"] - df["ride_distance"]) < 10]
    df.drop(["in_Lviv", "simple_distance"], axis=1, inplace=True)
    return df

def add_agg(df_in, df_out, group_by, name, aggs, add_name):
    if "mode" in aggs:
        aggs.remove("mode")
        _ = df_in.groupby(group_by)[name].agg(aggs)
        _["mode"] = df_in.groupby(group_by)[name].apply(lambda x: np.bincount(x).argmax())
    else:
        _ = df_in.groupby(group_by)[name].agg(aggs)
    _.columns = ["_".join([add_name, group_by, name, column]) for column in _.columns]
    return df_out.merge(_, left_index=True, right_index=True, how="left")

def make_aggs(df, name, days, LAST_DAY):
    df__ = pd.DataFrame(index=df[name].unique())

    df_ = df[(LAST_DAY - df.created_at).dt.days > 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "all")
    df__ = df__[df__[f"all_{name}_canceled_by_client_count"] > days]
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "all")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "all")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "all")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "all")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "all")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "all")
    
    df_ = df[(LAST_DAY - df.created_at).dt.days // 8 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "3_month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "3_month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "3_month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "3_month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "3_month")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 6 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "3_month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "3_month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "3_month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "3_month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "3_month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "3_month")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 4 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "month")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "month")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "month")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "month")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "month")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "month")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "month")
    df__[f"{name}_active"] = df__.index.isin(df_[name])

    df_ = df[(LAST_DAY - df.created_at).dt.days // 3 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "week")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "week")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "week")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "week")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "week")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "week")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "week")

    df_ = df[(LAST_DAY - df.created_at).dt.days // 2 == 0]
    df__ = add_agg(df_, df__, name, "canceled_by_client", ["mean", "median", "std", "count"], "day")
    df__ = add_agg(df_, df__, name, "canceled_by_driver", ["mean", "median", "std"], "day")
    df__ = add_agg(df_, df__, name, "ride_to_suburb", ["mean", "std"], "day")
    df__ = add_agg(df_, df__, name, 'ride_distance', ["mean", "std", "median", "min", "max"], "day")
    df__ = add_agg(df_, df__, name, 'dayofweek', ["median", "mode"], "day")
    df__ = add_agg(df_, df__, name, 'final_time', ["median", "mode"], "day")
    df__ = add_agg(df_, df__, name, 'is_weekend', ["median", "std", "mean", "mode"], "day")
    print(df__.shape)
    df__.fillna(0.0, inplace=True)
    return df__

In [4]:
%%time
df = clear_df(df)

LAST_DAY = df.created_at.max()
LAST_DAY

CPU times: user 3min 23s, sys: 208 ms, total: 3min 24s
Wall time: 3min 24s


In [5]:
%%time
df_time = make_df_time(df)
df = df.merge(df_time, how="left", left_index=True, right_index=True)

CPU times: user 1.02 s, sys: 88 ms, total: 1.11 s
Wall time: 1.1 s


In [6]:
# %%time
# df_driver = make_aggs(df, "driver_id", 30)
# %%time
# df_user = make_aggs(df, "user_id", 10)
# df = df.merge(df_driver, how="left", left_on="driver_id", right_index=True)
# df = df.merge(df_user, how="left", left_on="user_id", right_index=True)

In [ ]:
df.dropna(0, inplace=True)
df_cell = make_aggs(df, "cell_id", 26, LAST_DAY)
df = df[df.cell_id.isin(df_cell.index)]
df.to_csv("new_csv.csv")

(43, 133)


In [8]:
df_cell["all_cell_id_canceled_by_client_count"]

871e7689cffffff    100717
871e76882ffffff     67707
871e7689dffffff     76452
871e768b3ffffff      3651
871e7689bffffff     26383
871e76126ffffff     19012
871e76883ffffff    236483
871e76895ffffff      8428
871e76898ffffff     35606
871e76880ffffff     42195
871e76899ffffff     33689
871e76881ffffff     37953
871e7689affffff     33927
871e76124ffffff      2906
871e76134ffffff     12760
871e7689effffff     85399
871e76891ffffff     43746
871e76890ffffff      4678
871e76135ffffff      7406
871e76893ffffff     15215
871e7688affffff     16888
871e7688effffff     18577
871e76885ffffff      6301
871e76896ffffff       827
871e76c69ffffff       856
871e76892ffffff      2348
871e76c6dffffff      3400
871e768aaffffff      3310
871e7688cffffff       536
871e76888ffffff      1931
871e76886ffffff      1009
871e768abffffff       406
871e76122ffffff       351
871e76c68ffffff       309
871e76894ffffff       244
871e768b1ffffff       274
871e76c6cffffff       187
871e768a8ffffff        73
871e7688bfff

In [9]:
df.shape

(957478, 24)

In [10]:
center = list(df_cell.index)
center_size = len(center)

In [11]:
df_ = df[df["cell_id"].isin(center)]

In [12]:
from datetime import timedelta

from keras.utils import to_categorical

Using TensorFlow backend.


In [13]:
day = LAST_DAY.date()
day = day - timedelta(0)

In [22]:
day_nums = 30
X = []
y = np.zeros((24, day_nums, center_size))
for k in range(0, day_nums):
    day = LAST_DAY.date() - timedelta(k + 22)
    redundant_cols=["hour", "minute", "final_time"]

    df__time = make_df_time(pd.DataFrame({"created_at": [pd.Timestamp(day)]}))
    df__time.drop(redundant_cols, axis=1, inplace=True)
    df__time = np.int16(np.concatenate((to_categorical(df__time["month"], 12)[0],
                   to_categorical(df__time["dayofweek"], 7)[0],
                   df__time.drop(["month", "dayofweek"], axis=1).values[0])))
    
    
    res = make_aggs(df_, "cell_id", 0, day).values.reshape(1, -1)[0]
    X.append(np.concatenate((df__time, res)).reshape(1, -1))
    
    group = df_[df_.created_at.dt.date == day].groupby(["final_time", "cell_id"]).created_at.count()
    for i in range(24):
        for j, cell in enumerate(center):
            if i in group:
                if cell in group.loc[i]:
                    y[i][k][j] = group.loc[i, cell]

(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)
(43, 133)


In [23]:
X = np.array(X)
X.shape

(30, 1, 5745)

In [24]:
y.shape

(24, 30, 43)

In [25]:
from keras.layers import Dense, Flatten, LSTM, Input, BatchNormalization, Dropout
from keras.models import Model
import keras.backend as K 

In [46]:
inp = Input(tuple(X.shape[1:]))
lstm1 = LSTM(units=128, activation="relu", return_sequences=True)(inp)
# lstm2 = LSTM(units=128, activation="relu", return_sequences=True)(lstm1)
outs= []
for i in range(y.shape[0]):
    flt = Flatten()(lstm1)
    dns = Dense(units=128, activation="relu")(flt)
    bn = BatchNormalization()(dns)
    outs.append(Dense(center_size, activation="relu")(bn))
model = Model(inp, outs)

In [47]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 1, 5745)      0                                            
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 1, 128)       3007488     input_5[0][0]                    
__________________________________________________________________________________________________
flatten_97 (Flatten)            (None, 128)          0           lstm_7[0][0]                     
__________________________________________________________________________________________________
flatten_98 (Flatten)            (None, 128)          0           lstm_7[0][0]                     
__________________________________________________________________________________________________
flatten_99

In [48]:
from keras.metrics import mean_squared_logarithmic_error

In [49]:
model = Model(inp, outs)
def rmsle(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(y_pred + 1.) - K.log(y_true + 1.)), axis=-1)) 

model.compile(optimizer = "adam", loss = mean_squared_logarithmic_error, metrics =["accuracy"])

In [50]:
model.fit(X, list(y), batch_size=2, epochs=10000)

Epoch 1/10000
30/30 [==============================] - 19s 626ms/step - loss: 49.7594 - dense_194_loss: 2.2443 - dense_196_loss: 1.7175 - dense_198_loss: 1.3085 - dense_200_loss: 0.8866 - dense_202_loss: 0.7293 - dense_204_loss: 0.9202 - dense_206_loss: 0.8346 - dense_208_loss: 0.9751 - dense_210_loss: 2.1409 - dense_212_loss: 2.8631 - dense_214_loss: 2.9755 - dense_216_loss: 2.6277 - dense_218_loss: 2.1128 - dense_220_loss: 2.0660 - dense_222_loss: 2.1180 - dense_224_loss: 2.1448 - dense_226_loss: 2.3068 - dense_228_loss: 2.5740 - dense_230_loss: 2.9851 - dense_232_loss: 2.8780 - dense_234_loss: 2.6863 - dense_236_loss: 2.4457 - dense_238_loss: 2.6098 - dense_240_loss: 2.6089 - dense_194_acc: 0.0000e+00 - dense_196_acc: 0.0000e+00 - dense_198_acc: 0.0000e+00 - dense_200_acc: 0.4000 - dense_202_acc: 0.0000e+00 - dense_204_acc: 0.0000e+00 - dense_206_acc: 0.1333 - dense_208_acc: 0.0000e+00 - dense_210_acc: 0.1667 - dense_212_acc: 0.0000e+00 - dense_214_acc: 0.0000e+00 - dense_216_acc: 0

30/30 [==============================] - 1s 49ms/step - loss: 37.0116 - dense_194_loss: 1.6787 - dense_196_loss: 1.2638 - dense_198_loss: 0.9963 - dense_200_loss: 0.6931 - dense_202_loss: 0.6046 - dense_204_loss: 0.6912 - dense_206_loss: 0.6388 - dense_208_loss: 0.6901 - dense_210_loss: 1.5661 - dense_212_loss: 2.0908 - dense_214_loss: 2.2125 - dense_216_loss: 1.9442 - dense_218_loss: 1.5251 - dense_220_loss: 1.5918 - dense_222_loss: 1.5396 - dense_224_loss: 1.5698 - dense_226_loss: 1.6901 - dense_228_loss: 1.8777 - dense_230_loss: 2.2242 - dense_232_loss: 2.1915 - dense_234_loss: 2.0186 - dense_236_loss: 1.8506 - dense_238_loss: 1.9283 - dense_240_loss: 1.9341 - dense_194_acc: 0.0333 - dense_196_acc: 0.1000 - dense_198_acc: 0.5000 - dense_200_acc: 0.5000 - dense_202_acc: 0.4667 - dense_204_acc: 0.4000 - dense_206_acc: 0.3667 - dense_208_acc: 0.0667 - dense_210_acc: 0.2667 - dense_212_acc: 0.1333 - dense_214_acc: 0.1667 - dense_216_acc: 0.0333 - dense_218_acc: 0.4667 - dense_220_acc: 0

30/30 [==============================] - 1s 41ms/step - loss: 34.1875 - dense_194_loss: 1.5093 - dense_196_loss: 1.1801 - dense_198_loss: 0.9118 - dense_200_loss: 0.6578 - dense_202_loss: 0.5956 - dense_204_loss: 0.5525 - dense_206_loss: 0.5840 - dense_208_loss: 0.6458 - dense_210_loss: 1.4066 - dense_212_loss: 1.9299 - dense_214_loss: 2.0539 - dense_216_loss: 1.7701 - dense_218_loss: 1.4099 - dense_220_loss: 1.4398 - dense_222_loss: 1.4452 - dense_224_loss: 1.4870 - dense_226_loss: 1.5822 - dense_228_loss: 1.7574 - dense_230_loss: 2.0705 - dense_232_loss: 2.0251 - dense_234_loss: 1.8592 - dense_236_loss: 1.7293 - dense_238_loss: 1.7966 - dense_240_loss: 1.7878 - dense_194_acc: 0.4000 - dense_196_acc: 0.0333 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.3667 - dense_206_acc: 0.5000 - dense_208_acc: 0.2000 - dense_210_acc: 0.1667 - dense_212_acc: 0.1667 - dense_214_acc: 0.1000 - dense_216_acc: 0.1667 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 40ms/step - loss: 32.8764 - dense_194_loss: 1.4193 - dense_196_loss: 1.1563 - dense_198_loss: 0.8846 - dense_200_loss: 0.6592 - dense_202_loss: 0.5738 - dense_204_loss: 0.2802 - dense_206_loss: 0.5604 - dense_208_loss: 0.6440 - dense_210_loss: 1.3568 - dense_212_loss: 1.8878 - dense_214_loss: 1.9531 - dense_216_loss: 1.6936 - dense_218_loss: 1.3947 - dense_220_loss: 1.4182 - dense_222_loss: 1.3919 - dense_224_loss: 1.4381 - dense_226_loss: 1.5320 - dense_228_loss: 1.6891 - dense_230_loss: 2.0102 - dense_232_loss: 1.9700 - dense_234_loss: 1.7987 - dense_236_loss: 1.6646 - dense_238_loss: 1.7663 - dense_240_loss: 1.7337 - dense_194_acc: 0.5000 - dense_196_acc: 0.4333 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.5333 - dense_206_acc: 0.6333 - dense_208_acc: 0.3333 - dense_210_acc: 0.2333 - dense_212_acc: 0.3333 - dense_214_acc: 0.1333 - dense_216_acc: 0.3333 - dense_218_acc: 0.5000 - dense_220_acc: 0

30/30 [==============================] - 1s 45ms/step - loss: 32.1296 - dense_194_loss: 1.3642 - dense_196_loss: 1.1228 - dense_198_loss: 0.8230 - dense_200_loss: 0.6342 - dense_202_loss: 0.5521 - dense_204_loss: 0.2528 - dense_206_loss: 0.5529 - dense_208_loss: 0.6314 - dense_210_loss: 1.2889 - dense_212_loss: 1.8660 - dense_214_loss: 1.9092 - dense_216_loss: 1.6916 - dense_218_loss: 1.3850 - dense_220_loss: 1.4153 - dense_222_loss: 1.3391 - dense_224_loss: 1.4500 - dense_226_loss: 1.4913 - dense_228_loss: 1.6629 - dense_230_loss: 1.9597 - dense_232_loss: 1.9132 - dense_234_loss: 1.7630 - dense_236_loss: 1.6401 - dense_238_loss: 1.7340 - dense_240_loss: 1.6868 - dense_194_acc: 0.5000 - dense_196_acc: 0.5000 - dense_198_acc: 0.5000 - dense_200_acc: 0.4667 - dense_202_acc: 0.4667 - dense_204_acc: 0.8333 - dense_206_acc: 0.6333 - dense_208_acc: 0.3667 - dense_210_acc: 0.2000 - dense_212_acc: 0.3000 - dense_214_acc: 0.2000 - dense_216_acc: 0.3667 - dense_218_acc: 0.4667 - dense_220_acc: 0

...

30/30 [==============================] - 1s 37ms/step - loss: 4.3236 - dense_194_loss: 0.1554 - dense_196_loss: 0.1982 - dense_198_loss: 0.2228 - dense_200_loss: 0.2631 - dense_202_loss: 0.2450 - dense_204_loss: 0.1959 - dense_206_loss: 0.1681 - dense_208_loss: 0.1640 - dense_210_loss: 0.2347 - dense_212_loss: 0.2450 - dense_214_loss: 0.2436 - dense_216_loss: 0.1632 - dense_218_loss: 0.1381 - dense_220_loss: 0.1476 - dense_222_loss: 0.1412 - dense_224_loss: 0.1402 - dense_226_loss: 0.1442 - dense_228_loss: 0.1404 - dense_230_loss: 0.1518 - dense_232_loss: 0.1668 - dense_234_loss: 0.1503 - dense_236_loss: 0.1622 - dense_238_loss: 0.1808 - dense_240_loss: 0.1611 - dense_194_acc: 1.0000 - dense_196_acc: 1.0000 - dense_198_acc: 1.0000 - dense_200_acc: 0.9667 - dense_202_acc: 0.9667 - dense_204_acc: 0.8333 - dense_206_acc: 0.4000 - dense_208_acc: 0.8000 - dense_210_acc: 0.4000 - dense_212_acc: 0.5333 - dense_214_acc: 0.3333 - dense_216_acc: 0.4667 - dense_218_acc: 0.9667 - dense_220_acc: 0.

30/30 [==============================] - 1s 37ms/step - loss: 4.2979 - dense_194_loss: 0.1553 - dense_196_loss: 0.1989 - dense_198_loss: 0.2284 - dense_200_loss: 0.2634 - dense_202_loss: 0.2459 - dense_204_loss: 0.1884 - dense_206_loss: 0.1643 - dense_208_loss: 0.1591 - dense_210_loss: 0.2360 - dense_212_loss: 0.2566 - dense_214_loss: 0.2517 - dense_216_loss: 0.1739 - dense_218_loss: 0.1374 - dense_220_loss: 0.1486 - dense_222_loss: 0.1377 - dense_224_loss: 0.1412 - dense_226_loss: 0.1380 - dense_228_loss: 0.1334 - dense_230_loss: 0.1483 - dense_232_loss: 0.1711 - dense_234_loss: 0.1476 - dense_236_loss: 0.1485 - dense_238_loss: 0.1703 - dense_240_loss: 0.1539 - dense_194_acc: 1.0000 - dense_196_acc: 1.0000 - dense_198_acc: 1.0000 - dense_200_acc: 0.9667 - dense_202_acc: 0.9667 - dense_204_acc: 0.8333 - dense_206_acc: 0.4000 - dense_208_acc: 0.8000 - dense_210_acc: 0.3000 - dense_212_acc: 0.4000 - dense_214_acc: 0.2333 - dense_216_acc: 0.4667 - dense_218_acc: 0.9667 - dense_220_acc: 0.

30/30 [==============================] - 1s 41ms/step - loss: 4.3276 - dense_194_loss: 0.1498 - dense_196_loss: 0.2040 - dense_198_loss: 0.2275 - dense_200_loss: 0.2634 - dense_202_loss: 0.2464 - dense_204_loss: 0.1936 - dense_206_loss: 0.1722 - dense_208_loss: 0.1576 - dense_210_loss: 0.2441 - dense_212_loss: 0.2591 - dense_214_loss: 0.2542 - dense_216_loss: 0.1672 - dense_218_loss: 0.1415 - dense_220_loss: 0.1437 - dense_222_loss: 0.1439 - dense_224_loss: 0.1401 - dense_226_loss: 0.1439 - dense_228_loss: 0.1338 - dense_230_loss: 0.1493 - dense_232_loss: 0.1666 - dense_234_loss: 0.1491 - dense_236_loss: 0.1510 - dense_238_loss: 0.1705 - dense_240_loss: 0.1550 - dense_194_acc: 1.0000 - dense_196_acc: 1.0000 - dense_198_acc: 1.0000 - dense_200_acc: 0.9667 - dense_202_acc: 0.9667 - dense_204_acc: 0.8333 - dense_206_acc: 0.4000 - dense_208_acc: 0.8000 - dense_210_acc: 0.3000 - dense_212_acc: 0.4333 - dense_214_acc: 0.2000 - dense_216_acc: 0.4667 - dense_218_acc: 0.9667 - dense_220_acc: 0.

30/30 [==============================] - 1s 43ms/step - loss: 4.2756 - dense_194_loss: 0.1511 - dense_196_loss: 0.1976 - dense_198_loss: 0.2235 - dense_200_loss: 0.2619 - dense_202_loss: 0.2514 - dense_204_loss: 0.1929 - dense_206_loss: 0.1726 - dense_208_loss: 0.1611 - dense_210_loss: 0.2297 - dense_212_loss: 0.2465 - dense_214_loss: 0.2437 - dense_216_loss: 0.1673 - dense_218_loss: 0.1412 - dense_220_loss: 0.1447 - dense_222_loss: 0.1420 - dense_224_loss: 0.1359 - dense_226_loss: 0.1412 - dense_228_loss: 0.1320 - dense_230_loss: 0.1494 - dense_232_loss: 0.1677 - dense_234_loss: 0.1490 - dense_236_loss: 0.1494 - dense_238_loss: 0.1708 - dense_240_loss: 0.1532 - dense_194_acc: 1.0000 - dense_196_acc: 1.0000 - dense_198_acc: 1.0000 - dense_200_acc: 0.9667 - dense_202_acc: 0.9667 - dense_204_acc: 0.8333 - dense_206_acc: 0.5667 - dense_208_acc: 0.8000 - dense_210_acc: 0.3000 - dense_212_acc: 0.5000 - dense_214_acc: 0.3333 - dense_216_acc: 0.4667 - dense_218_acc: 0.9667 - dense_220_acc: 0.

KeyboardInterrupt: 